<a href="https://colab.research.google.com/github/Ihoold/pixelgen/blob/master/funit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from unittest import mock
import os
import numpy as np
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from tensorflow.python.estimator import estimator
from google.colab import drive, auth

In [0]:
R, C = 4, 3
EXAMPLES = R * C
    
CHANNELS = 4
LATENT_DIM = 128
ADD_NOISE_TO_EXAMPLE = False

DROPOUT_PROB = 0.5
ALPHA = 0.2
BATCH_SIZE = 1024
EPOCHS = 10000
EVAL_EPOCHS = 500
G_LR = 0.0002
D_LR = 0.0001
KERNEL_SIZE = 4
NUM_CLASSES = 21

RUN_NAME = 'FUNIT_2'

data_train_file = 'gs://tputestingmnist/datasets/dataset_train_funit.tfrecords'
data_test_file = 'gs://tputestingmnist/datasets/dataset_test_funit.tfrecords'
MODEL_DIR = 'gs://tputestingmnist/{}/'.format(RUN_NAME)
GOOGLE_DRIVE_DIR = '/content/gdrive/My Drive/Programowanie/PixelGen/{}'.format(RUN_NAME)
TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])

In [0]:
#################################### SETUP #####################################

def setup():
    drive.mount('/content/gdrive')
    auth.authenticate_user()


def upload_credentials():
    # Upload credentials to TPU.
    with tf.Session(TF_MASTER) as sess:    
        with open('/content/adc.json', 'r') as f:
            auth_info = json.load(f)
        tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)

In [0]:
################################# DATA INPUT ###################################

def parser(serialized_example):
        """Parses a single example into image and label tensors."""
        features = tf.parse_single_example(
            serialized_example,
            features={
                'image_content': tf.FixedLenFeature([], tf.string),
                'image_style': tf.FixedLenFeature([], tf.string),
                'label': tf.FixedLenFeature([], tf.int64) 
            })

        content_image = tf.decode_raw(features['image_content'], tf.uint8)
        content_image.set_shape([48 * 48 * 4])
        content_image = tf.reshape(content_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        content_image = tf.cast(content_image, dtype=tf.float32) / 127.5 - 1

        style_image = tf.decode_raw(features['image_style'], tf.uint8)
        style_image.set_shape([48 * 48 * 4])
        style_image = tf.reshape(style_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        style_image = tf.cast(style_image, dtype=tf.float32) / 127.5 - 1
        
        label = features['label']

        return style_image, content_image, label


def make_input_fn(is_training=True):
    def input_fn(params):
        batch_size = params['batch_size']
        dataset = tf.data.TFRecordDataset(data_train_file).map(parser).cache().shuffle(batch_size)
        if is_training:
            dataset = dataset.repeat()
        style_images, content_images, labels = \
            dataset.prefetch(batch_size).batch(batch_size, drop_remainder=True).make_one_shot_iterator().get_next()

        features = {
            'style_images': style_images,
            'content_images': content_images
        }
        return features, labels
    return input_fn


def predict_input_fn(params):
    batch_size = params['batch_size']
    dataset = tf.data.TFRecordDataset(data_test_file).map(parser).cache().shuffle(batch_size)
    style_images, content_images, _ = dataset.prefetch(batch_size).batch(batch_size, drop_remainder=True).make_one_shot_iterator().get_next()
    
    features = {
        'style_images': style_images,
        'content_images': content_images,
    }
    return features, None


In [0]:
############################### DATA SAVEING ###################################
 
def images_to_zero_one(images):
        return np.clip(np.array(images) * 0.5 + 0.5, 0., 1.)


def save_imgs(epoch, images):
    if not os.path.exists(GOOGLE_DRIVE_DIR):
        os.mkdir(GOOGLE_DRIVE_DIR)

    # Rescale images to 0 - 1
    images = images_to_zero_one(images)
    fig, axs = plt.subplots(R, C, figsize=(20,20))

    for i in range(R):
        for j in range(C):
            axs[i,j].imshow(images[C*i + j])
            axs[i,j].axis('off')

    fig.savefig(os.path.join(GOOGLE_DRIVE_DIR, '{}.png'.format(epoch)))
    plt.close()

In [0]:
################################## MODEL #######################################

def _relu(x):
    return tf.nn.relu(x)


def _leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=ALPHA)


def _batch_norm(x, is_training, name):
    return tf.layers.batch_normalization(x, momentum=0.8, epsilon=1e-5, 
                                         training=is_training, name=name)


def _instance_norm(x):
    return tf.contrib.layers.instance_norm(x)


def _make_adain_norm(style_mean, style_var):
    def _adain_norm(x):
        mean, var = tf.nn.moments(x, axes=[0])
        return tf.nn.batch_normalization(x, mean, var, style_mean, style_var)
    return _adain_norm


def _dense(x, neurons, name, activation=None):
    return tf.layers.dense(x, neurons, name=name, activation=activation,
                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def _conv2d(x, filters, stride, name, activation=None):
    return tf.layers.conv2d(x, filters, [KERNEL_SIZE, KERNEL_SIZE], 
                            strides=[stride, stride], activation=activation,
                            padding='same', name=name,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

def _pooling(x, stride, index):
    return tf.layers.average_pooling2d(x, pool_size=stride, strides=stride, name='pool_{}'.format(index))

def _deconv2d(x, filters, stride, name, activation=None):
    return tf.layers.conv2d_transpose(x, filters, [KERNEL_SIZE, KERNEL_SIZE],
                                      strides=[stride, stride], activation=activation,
                                      padding='same', name=name,
                                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def convolution_block(x, filters, resize_factor, index, activation=_relu, activation_first=False, normalization=None):
    if activation and activation_first:
        x = activation(x)
    x = _conv2d(x, filters=filters, stride=resize_factor, activation=None, name='conv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation and not activation_first:
        x = activation(x)
    return x


def deconvolution_block(x, filters, resize_factor, index, activation=_relu, normalization=None):
    x = _deconv2d(x, filters=filters, stride=resize_factor, activation=None, name='deconv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def fully_connected_block(x, neurons, index, activation=_relu, normalization=None):
    x = _dense(x, neurons=neurons, activation=None, name='fc_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def residual_block(x, filters_in, filters_out, index, activation=_relu, activation_first=False, normalization=None):
    forwarded_x = convolution_block(x, filters_in, 1, index='res_con1_{}'.format(index), activation_first=activation_first, activation=activation)
    forwarded_x = convolution_block(forwarded_x, filters_out, 1, index='res_con2_{}'.format(index), activation_first=activation_first, activation=activation)
    if filters_in != filters_out:
        x = convolution_block(x, filters_out, 1, index='res_con3_{}'.format(index), activation_first=activation_first, activation=activation)
    x = tf.add(forwarded_x, x)
    return x


def adain_residual_block(x, mean, var, filters_in, filters_out, index, activation=_relu, activation_first=False):
    normalization = _make_adain_norm(mean, var)
    return residual_block(x, filters_in, filters_out, 'adain_{}'.format(index), activation, activation_first, normalization)


def content_encoder(x):
    x = convolution_block(x, 64, resize_factor=1, normalization=_instance_norm, index='con1')
    x = convolution_block(x, 128, resize_factor=2, normalization=_instance_norm, index='con2')
    x = convolution_block(x, 256, resize_factor=2, normalization=_instance_norm, index='con3')
    x = convolution_block(x, 512, resize_factor=2, normalization=_instance_norm, index='con4')
    
    x = residual_block(x, 512, 512, index='res1', normalization=_instance_norm)
    x = residual_block(x, 512, 512, index='res2', normalization=_instance_norm)
    return x


def decoder(content_latent, class_latent):    
    class_latent = fully_connected_block(class_latent, 256, 'dec11')
    class_latent = fully_connected_block(class_latent, 256, 'dec12')
    class_latent = fully_connected_block(class_latent, 256, 'dec13')
    mean, var = tf.nn.moments(class_latent, axes=[0])
    
    x = adain_residual_block(content_latent, mean, var, 512, 512, index='res3')
    x = adain_residual_block(x, mean, var, 512, 512, index='res3')

    x = deconvolution_block(x, 256, resize_factor=2, normalization=_instance_norm, index='dec1')
    x = deconvolution_block(x, 128, resize_factor=2, normalization=_instance_norm, index='dec2')
    x = deconvolution_block(x, 64, resize_factor=2, normalization=_instance_norm, index='dec3')
    x = deconvolution_block(x, CHANNELS, resize_factor=1, normalization=_instance_norm, index='dec4', activation=tf.tanh)
    
    return x


def class_encoder(x):
    x = convolution_block(x, 64, resize_factor=1, index='cls1')
    x = convolution_block(x, 128, resize_factor=2, index='cls2')
    x = convolution_block(x, 256, resize_factor=2, index='cls3')
    x = convolution_block(x, 512, resize_factor=2, index='cls4')
    x = convolution_block(x, 1024, resize_factor=2, index='cls5')
    x = _pooling(x, 3, index='cls6')
    return x

class Funit:

    @staticmethod
    def discriminator(x, label, scope='Discriminator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            x = convolution_block(x, 64, 1, index='disc_11')
            
            x = residual_block(x, 64, 128, 'disc_12', _leaky_relu, activation_first=True)
            x = residual_block(x, 128, 128, 'disc_13', _leaky_relu, activation_first=True)
            x = _pooling(x, 2, 'disc_14')
            
            x = residual_block(x, 128, 256, 'disc_15', _leaky_relu, activation_first=True)
            x = residual_block(x, 256, 256, 'disc_16', _leaky_relu, activation_first=True)
            x = _pooling(x, 2, 'disc_17')
            
            x = residual_block(x, 256, 512, 'disc_18', _leaky_relu, activation_first=True)
            x = residual_block(x, 512, 512, 'disc_19', _leaky_relu, activation_first=True)
            x = _pooling(x, 2, 'disc_20')
            
            x = residual_block(x, 512, 1024, 'disc_21', _leaky_relu, activation_first=True)
            x = residual_block(x, 1024, 1024, 'disc_22', _leaky_relu, activation_first=True)
            x = _pooling(x, 2, 'disc_23')
            
            x = residual_block(x, 1024, 1024, 'disc_24', _leaky_relu, activation_first=True)
            x = residual_block(x, 1024, 1024, 'disc_25', _leaky_relu, activation_first=True)
            x = _pooling(x, 3, 'disc_26')
            
            x = convolution_block(x, NUM_CLASSES, 1, index='disc_26', activation=_leaky_relu, activation_first=True)
            x = tf.squeeze(x)
            label = tf.expand_dims(label, axis=1)
            x = tf.gather_nd(x, label, batch_dims=1)
            
            return x

    @staticmethod
    def generator(image_content, image_style, scope='Generator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            content_latent = content_encoder(image_content)
            class_latent = class_encoder(image_style)
            
            x = decoder(content_latent, class_latent)
            return x

In [0]:
################################ MODEL FUN #####################################
def make_model_fn(model):

    def model_fn(features, labels, mode, params):
        # PREDICT #
        if mode == tf.estimator.ModeKeys.PREDICT:
            content_images = features['content_images']
            style_images = features['style_images']
            generated_images = model.generator(content_images, style_images)
            predictions = {
                'generated_images': generated_images,
                'content_images': content_images,
                'style_images': style_images
            }
            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, predictions=predictions)

        content_images = features['content_images']
        style_images = features['style_images']
        generated_images = model.generator(content_images, style_images)

        # Discriminator loss
        
        d_on_data_logits = tf.squeeze(model.discriminator(style_images, labels))
        d_on_data_labels = tf.ones_like(d_on_data_logits)

        d_on_g_logits = tf.squeeze(model.discriminator(generated_images, labels))
        d_on_g_labels = tf.zeros_like(d_on_g_logits)

        d_loss = tf.contrib.gan.losses.wargs.modified_discriminator_loss(
            discriminator_real_outputs=d_on_data_logits,
            discriminator_gen_outputs=d_on_g_logits,
            reduction=tf.losses.Reduction.NONE,
            label_smoothing=0.2
        )

        # Generator loss
        g_loss = tf.contrib.gan.losses.wargs.modified_generator_loss(
            discriminator_gen_outputs=d_on_g_logits,
            reduction=tf.losses.Reduction.NONE
        )

        # TRAIN #
        if mode == tf.estimator.ModeKeys.TRAIN:
            d_loss = tf.reduce_mean(d_loss)
            g_loss = tf.reduce_mean(g_loss)
            d_optimizer = tf.train.AdamOptimizer(learning_rate=D_LR, beta1=0.5)
            g_optimizer = tf.train.AdamOptimizer(learning_rate=G_LR, beta1=0.5)

            d_optimizer = tf.contrib.tpu.CrossShardOptimizer(d_optimizer)
            g_optimizer = tf.contrib.tpu.CrossShardOptimizer(g_optimizer)
         
            with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                d_step = d_optimizer.minimize(d_loss, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                 scope='Discriminator'))
                g_step = g_optimizer.minimize(g_loss, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                 scope='Generator'))
                increment_step = tf.assign_add(tf.train.get_or_create_global_step(), 1)
                joint_op = tf.group([d_step, g_step, increment_step])

                a = tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=g_loss, train_op=joint_op)
                return a

        # EVAL #
        elif mode == tf.estimator.ModeKeys.EVAL:
            def _eval_metric_fn(d_loss, g_loss, d_real_labels, d_gen_lanels, d_real_logits, d_gen_logits):
                return {
                    'discriminator_loss': tf.metrics.mean(d_loss),
                    'generator_loss': tf.metrics.mean(g_loss),
                    'discriminator_real_accuracy': tf.metrics.accuracy(labels=d_real_labels, predictions=tf.math.round(tf.sigmoid(d_real_logits))),
                    'discriminator_gen_accuracy': tf.metrics.accuracy(labels=d_gen_lanels, predictions=tf.math.round(tf.sigmoid(d_gen_logits))),
                    'discriminator_real_outout': tf.metrics.mean(tf.math.round(tf.sigmoid(d_real_logits)).eval()),
                    'discriminator_gen_output': tf.metrics.mean(tf.math.round(tf.sigmoid(d_gen_logits)).eval()),
                }

            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=tf.reduce_mean(g_loss),
                                                   eval_metrics=(_eval_metric_fn, [d_loss, g_loss, d_on_data_labels,
                                                                                   d_on_g_labels, d_on_data_logits, d_on_g_logits]))
    return model_fn

In [0]:
################################ ESTIMATORS ####################################

def make_estimators(model, only_cpu=False):
    model_fn = make_model_fn(model)
    
    config = tf.contrib.tpu.RunConfig(
        master=TF_MASTER,
        save_checkpoints_steps=EVAL_EPOCHS,
        save_checkpoints_secs=None,
        save_summary_steps=EVAL_EPOCHS,
        model_dir=MODEL_DIR,
        keep_checkpoint_max=3,
        tpu_config=tf.contrib.tpu.TPUConfig(iterations_per_loop=EVAL_EPOCHS))

    if not only_cpu:
        # TPU-based estimator used for TRAIN and EVAL
        est = tf.contrib.tpu.TPUEstimator(
            model_fn=model_fn,
            use_tpu=True,
            config=config,
            train_batch_size=BATCH_SIZE,
            eval_batch_size=BATCH_SIZE)
    else:
        est = None

    # CPU-based estimator used for PREDICT (generating images)
    cpu_est = tf.contrib.tpu.TPUEstimator(
        model_fn=model_fn,
        use_tpu=False,
        config=config,
        predict_batch_size=EXAMPLES)
    
    return est, cpu_est

In [0]:
################################# TRAINING #####################################

def train(est, cpu_est):
    current_step = estimator._load_global_step_from_checkpoint_dir(MODEL_DIR)
    print('Starting training')

    while current_step < EPOCHS:
        next_checkpoint = int(min(current_step + EVAL_EPOCHS, EPOCHS))
        est.train(input_fn=make_input_fn(), max_steps=next_checkpoint)
        current_step = next_checkpoint
        print('Finished training step %d' % current_step)

        # Evaluation
        metrics = est.evaluate(input_fn=make_input_fn(False), steps=1)
        print('Finished evaluating')
        print(metrics)

        # Render some generated images
        generated_iter = cpu_est.predict(input_fn=predict_input_fn)
        images = [np.concatenate([p['content_images'], p['style_images'], p['generated_images']], axis=1) for p in generated_iter]
        save_imgs(str(current_step), images)
        print('Finished generating images')

In [0]:
def do_experiment():
    setup()
    upload_credentials()
    model = Funit()
    est, cpu_est = make_estimators(model)
    train(est, cpu_est)

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

try:
    do_experiment()
except Exception as e:
    print (e)
    pass

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


W0806 22:04:00.733345 140129358948224 estimator.py:1984] Estimator's model_fn (<function make_model_fn.<locals>.model_fn at 0x7f7245919ae8>) includes params argument, but params are not passed to Estimator.
I0806 22:04:00.735827 140129358948224 estimator.py:209] Using config: {'_model_dir': 'gs://tputestingmnist/FUNIT_2/', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f721bd512b0>, '_task_type': 'worker', '_task_id': 0, '_global_id

Starting training


I0806 22:04:01.883578 140129358948224 estimator.py:1145] Calling model_fn.
I0806 22:04:11.058784 140129358948224 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0806 22:04:11.298605 140129358948224 estimator.py:1147] Done calling model_fn.
I0806 22:04:11.300276 140129358948224 tpu_estimator.py:499] TPU job name tpu_worker
I0806 22:04:12.124225 140129358948224 monitored_session.py:240] Graph was finalized.
I0806 22:04:13.007988 140129358948224 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT_2/model.ckpt-500
I0806 22:04:37.063386 140129358948224 session_manager.py:500] Running local_init_op.
I0806 22:04:37.464433 140129358948224 session_manager.py:502] Done running local_init_op.
I0806 22:04:43.356651 140129358948224 basic_session_run_hooks.py:606] Saving checkpoints for 500 into gs://tputestingmnist/FUNIT_2/model.ckpt.
I0806 22:05:15.349372 140129358948224 util.py:98] Initialized dataset iterators in 0 seconds
I0806 22:05:15.351013 140129358948224 sessio